In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mitbih1/117.atr
/kaggle/input/mitbih1/103.dat
/kaggle/input/mitbih1/106.hea
/kaggle/input/mitbih1/113.dat
/kaggle/input/mitbih1/202.dat
/kaggle/input/mitbih1/217.dat
/kaggle/input/mitbih1/111.atr
/kaggle/input/mitbih1/208.hea
/kaggle/input/mitbih1/219.dat
/kaggle/input/mitbih1/104.atr
/kaggle/input/mitbih1/201.dat
/kaggle/input/mitbih1/201.atr
/kaggle/input/mitbih1/121.hea
/kaggle/input/mitbih1/219.atr
/kaggle/input/mitbih1/102.dat
/kaggle/input/mitbih1/116.hea
/kaggle/input/mitbih1/231.dat
/kaggle/input/mitbih1/111.hea
/kaggle/input/mitbih1/209.hea
/kaggle/input/mitbih1/122.atr
/kaggle/input/mitbih1/203.hea
/kaggle/input/mitbih1/212.atr
/kaggle/input/mitbih1/232.dat
/kaggle/input/mitbih1/100.hea
/kaggle/input/mitbih1/234.hea
/kaggle/input/mitbih1/214.dat
/kaggle/input/mitbih1/119.hea
/kaggle/input/mitbih1/228.dat
/kaggle/input/mitbih1/107.dat
/kaggle/input/mitbih1/108.hea
/kaggle/input/mitbih1/123.hea
/kaggle/input/mitbih1/121.dat
/kaggle/input/mitbih1/105.atr
/kaggle/in

In [2]:
import numpy as np
import wfdb
import matplotlib.pyplot as plt
from scipy.signal import resample
import glob
import pywt

def WTfilt_1d(sig):
    """
    对信号进行小波变换滤波
    :param sig: 输入信号，1-d array
    :return: 小波滤波后的信号，1-d array
    """
    coeffs = pywt.wavedec(sig, 'db6', level=9)
    coeffs[-1] = np.zeros(len(coeffs[-1]))
    coeffs[-2] = np.zeros(len(coeffs[-2]))
    coeffs[0] = np.zeros(len(coeffs[0]))
    sig_filt = pywt.waverec(coeffs, 'db6')
    return sig_filt    


# -------------------------心拍截取-------------------
def heartbeat(file0):
    '''
    file0:下载的MITAB数据
    
    '''
    N_Seg = []; SVEB_Seg = [];  VEB_Seg = []; F_Seg = [] ; Q_Seg = [];
    file = list(set(file0))
    
    for f in range(len(file)):
        annotation = wfdb.rdann(panth+file[f][-7:-4],'atr')
        record_name = annotation.record_name    # 读取记录名称
        Record = wfdb.rdsamp(panth+record_name)[0][:, 0]  # 一般只取一个导联
        record = WTfilt_1d(Record)         # 小波去噪
        label = annotation.symbol  # 心拍标签列表
        label_index = annotation.sample   # 标签索引列表
        for j in range(len(label_index)):
            if label_index[j] >= 144 and (label_index[j]+180) <= 650000:
                if label[j] == 'N' or label[j] == '.' or label[j] == 'L' or label[j] == 'R' or label[j] == 'e' or label[j] == 'j':
                    Seg = record[label_index[j]-144:label_index[j]+180]  # R峰的前0.4s和后0.5s
                    N_Seg.append(Seg)
                    
                if label[j] == 'A' or label[j] == 'a' or label[j] == 'J' or label[j] == 'S':
                    
                    Seg = record[label_index[j]-144:label_index[j]+180]
                    SVEB_Seg.append(Seg)
                    
                if label[j] == 'V' or label[j] == 'E':
                   
                    Seg = record[label_index[j]-144:label_index[j]+180]
                    VEB_Seg.append(Seg)
                    
                if label[j] == 'F':
                    
                    Seg = record[label_index[j]-144:label_index[j]+180]
                    F_Seg.append(Seg)

                if label[j] == '/' or label[j] == 'f' or label[j] == 'Q':
                    
                    Seg = record[label_index[j]-144:label_index[j]+180]
                    Q_Seg.append(Seg)
                    
    N_segement = np.array(N_Seg)
    SVEB_segement = np.array(SVEB_Seg)
    VEB_segement = np.array(VEB_Seg)
    F_segement = np.array(F_Seg)
    Q_segement = np.array(Q_Seg)
    
    label_N = np.zeros(N_segement.shape[0])
    label_SVEB = np.ones(SVEB_segement.shape[0])
    label_VEB = np.ones(VEB_segement.shape[0])*2
    label_F = np.ones(F_segement.shape[0])*3
    label_Q = np.ones(Q_segement.shape[0])*4
                    
    Data = np.concatenate((N_segement, SVEB_segement, VEB_segement, F_segement, Q_segement), axis=0)
    Label = np.concatenate((label_N, label_SVEB, label_VEB, label_F, label_Q), axis=0)
    
    return Data, Label

#-----------------------心拍截取和保存---------------------
#建议一次性截取和保存，不需要重复操作，下次训练和测试的时候，直接load
panth='/kaggle/input/mitbih1/'
file = glob.glob(panth+'*.hea')
Data, Label=heartbeat(file)

Data=np.save('/kaggle/working/'+'Data',Data)
Label=np.save('/kaggle/working/'+'Label',Label)

In [3]:
import keras
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, optimizers, datasets, Sequential
from tensorflow.keras.layers import Dropout, Bidirectional, LSTM, MaxPooling1D, Conv1D, Conv2D, AveragePooling1D, MaxPool1D, add, Flatten, Dense, Concatenate, Activation, Input
from keras import regularizers, Model
import os

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
tf.random.set_seed(2345)

def preprocess(x, y):
    """
    x is a simple image, not a batch
    """
    x = tf.cast(x, dtype=tf.float32)
    y = tf.cast(y, dtype=tf.int32)
    y = tf.one_hot(y, depth=5)
    return x, y




def folds(x, y, folds, num, start_index, end_index):

    part = int(num/folds)

    # 分割数据集;data1,data2分别为数据和标签
    x_test = x[start_index * part:end_index * part]  # 数据刚好可以做k折交叉验证。
    if start_index == 0:
        x_train = x[end_index * part:]
    else:
        x_train0 = x[0:start_index * part]
        x_train1 = x[end_index * part:]
        x_train = np.concatenate((x_train0, x_train1), axis=0)

    # 分割标签
    y_test = y[start_index * part:end_index * part]  # 数据刚好可以做k折交叉验证。
    if start_index == 0:
        y_train = y[end_index * part:]
    else:
        y_train0 = y[0:start_index * part]
        y_train1 = y[end_index * part:]
        y_train = np.concatenate((y_train0, y_train1), axis=0)

    print('fold:', folds, 'test:', start_index, ' train: other')

    return x_train, x_test, y_train, y_test  # 每一折的训练和测试



class mylstm(keras.Model):
    def __init__(self):
        super(mylstm, self).__init__()

        self.lstm = Sequential()
        self.lstm.add(Bidirectional(LSTM(64, dropout=0.1, return_sequences=True)))
        self.lstm.add(Bidirectional(LSTM(32, dropout=0.1, return_sequences=True)))


        self.outlayer = Sequential([
            layers.Dense(128, activation='relu'),
            Dropout(0.2),
            layers.Dense(5, activation='softmax')
            ])

    def call(self, inputs, training=None, mask=None):

        x = self.lstm(inputs)
        
        x = Flatten()(x)

        x = self.outlayer(x, training)

        return x

#导入数据
data = np.load('/kaggle/working/Data.npy')
label = np.load('/kaggle/working/Label.npy')

data = tf.expand_dims(data, axis=2)
db_num = int(data.shape[0])

idx = tf.range(db_num)
idx = tf.random.shuffle(idx)
x, y = tf.gather(data, idx[:]), tf.gather(label, idx[:])

fold = 5  # 5折交叉验证
batchsz = 128
# acc_avarage = []
# loss_avarage = []

for i in range(0, fold):
    x_train, x_test, y_train, y_test = folds(x, y, fold, db_num, i, i+1)

    db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    db_train = db_train.map(preprocess).shuffle(90000).batch(batchsz)

    db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    db_test = db_test.map(preprocess).batch(batchsz)

    model = mylstm()
    model.build(input_shape=(None,324,1))
    model.summary()

    model.compile(optimizer=optimizers.Adam(),
                  loss='categorical_crossentropy',
                  metrics=['categorical_accuracy'])

    model.fit(db_train, epochs=30, validation_data=db_test, validation_freq=1)
    loss, accuracy = model.evaluate(db_test)
    
    # 一次分类的训练
#     if i == 0:
#         break


    

fold: 5 test: 0  train: other
Model: "mylstm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 324, 64)           75008     
_________________________________________________________________
sequential_1 (Sequential)    (None, 5)                 2654981   
Total params: 2,729,989
Trainable params: 2,729,989
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
685/685 [==============================] - 60s 75ms/step - loss: 0.1392 - categorical_accuracy: 0.9621 - val_loss: 0.0582 - val_categorical_accuracy: 0.9843
Epoch 2/30
685/685 [==============================] - 53s 73ms/step - loss: 0.0618 - categorical_accuracy: 0.9823 - val_loss: 0.0431 - val_categorical_accuracy: 0.9881
Epoch 3/30
685/685 [==============================] - 53s 73ms/step - loss: 0.0473 - categorical_accuracy: 0.9860 - val_loss: 0.0414 - val_categor